In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
DUR = 2.0 * YR
FMIN = 1 / DUR
DF = FMIN / 2.0
fobs = FMIN
fobs_edges = FMIN + np.array([-DF, +DF])
print(fobs_edges*YR, "1/yr")

In [ ]:
NREALS = 30
hard = holo.evolution.Hard_GW()
sam = holo.sam.Semi_Analytic_Model()

In [ ]:
fobs_orb_edges = fobs_edges / 2.0
edges, dnum = sam.dynamic_binary_number(hard, fobs=fobs_orb_edges)
number_0 = holo.utils._integrate_grid_differential_number(edges, dnum, freq=True)
number_1 = holo.utils._integrate_grid_differential_number(edges, dnum, freq=False)
number_1 = holo.utils.trapz(number_1/edges[3], edges[3], axis=3, cumsum=False)

ratio = np.nan_to_num(number_1/number_0)
print(kale.utils.stats_str(ratio))

In [ ]:
cents = [kale.utils.midpoints(ee, log=False) for ee in edges]
cgrid = np.meshgrid(*cents, indexing='ij')
egrid = np.meshgrid(*edges, indexing='ij')

In [ ]:
def hs_from_grid(grid):
    mc = utils.chirp_mass(*utils.m1m2_from_mtmr(grid[0], grid[1]))
    dc = cosmo.comoving_distance(grid[2]).cgs.value
    # These should be *orbital*-frequencies
    fr = utils.frst_from_fobs(grid[3], grid[2])
    hs = utils.gw_strain_source(mc, dc, fr)
    return hs

hs_cents = hs_from_grid(cgrid)
hs_edges = hs_from_grid(egrid)
hs_edge_cents = kale.utils.midpoints(hs_edges, axis=None)

ratio = hs_cents/hs_edge_cents
print(kale.utils.stats_str(ratio))

In [ ]:
hs_weight = dnum * (hs_edges ** 2)
print(utils.stats(hs_weight))
hs_weight = np.sqrt(utils._integrate_grid_differential_number(edges, hs_weight, freq=True) / number_0)
hs_weight = np.nan_to_num(hs_weight)
ratio = hs_weight / hs_edge_cents
print(kale.utils.stats_str(ratio))

In [ ]:
# gwb_ref = sam.gwb(fobs_edges, realize=NREALS, hard=hard)
gwb_test = sam.gwb(fobs_edges, realize=NREALS, hard=hard)

In [ ]:
gwb_test.shape

In [ ]:
gff = np.zeros(NREALS)
gwf = np.zeros(NREALS)
gwb = np.zeros(NREALS)

for ii in range(NREALS):
    vals, weights, edges, dens, mass = holo.sam.sample_sam_with_hardening(sam, hard, fobs=fobs_edges, sample_threshold=1e2, poisson_inside=True, poisson_outside=True)
    gff[ii], gwf[ii], gwb[ii] = holo.gravwaves._gws_from_samples(vals, weights, 2*fobs_edges)

In [ ]:
tot = np.sqrt(gwb**2 + gwf**2)
aves = []
meds = []
for val in [gwb_ref, tot]:
    sel = np.percentile(val, [25, 75])
    sel = (sel[0] < val) & (val < sel[1])
    ave = np.mean(val[sel])
    med = np.median(val[sel])
    std = np.std(val[sel])
    aves.append(ave)
    meds.append(med)

for vals in [aves, meds]:
    diff = (vals[1] - vals[0]) / vals[0]
    print(f"{diff:.4f}")

In [ ]:
kw = dict(density=True, hist=True, carpet=True, confidence=True, quantiles=[0.5])
kale.dist1d(gwb_ref, **kw)
kale.dist1d(gwb_test, color='r', **kw)
kale.dist1d(np.sqrt(gwf**2 + gwb**2), color='k', **kw)